In [1]:
import cv2
import numpy as np
np.set_printoptions(threshold=np.nan)
from matplotlib import pyplot as plt
%matplotlib inline
from math import ceil
from math import pow
from math import sqrt
from copy import deepcopy

In [2]:
image = cv2.imread('gray2_encoded.jpg',cv2.IMREAD_GRAYSCALE)
image_dash = cv2.imread('gray2_encoded.jpg',cv2.IMREAD_GRAYSCALE)

In [3]:
x_checkpoint_pixels = []
y_checkpoint_pixels = []
number_of_blocks = 16
#Assuming the image to be square shaped
step_size = image_dash.shape[0]//int(sqrt(number_of_blocks))
for checkpoint in range(0,image_dash.shape[0],step_size):
    x_checkpoint_pixels.append(checkpoint)
    y_checkpoint_pixels.append(checkpoint)
x_checkpoint_pixels.append(image_dash.shape[0])
y_checkpoint_pixels.append(image_dash.shape[0])

print(x_checkpoint_pixels)

[0, 32, 64, 96, 128]


In [4]:
blockwise_pixels = []
for i in range(len(y_checkpoint_pixels)-1):
    for j in range(len(x_checkpoint_pixels)-1):
        blockwise_pixels.append(image_dash[x_checkpoint_pixels[j]:x_checkpoint_pixels[j+1],y_checkpoint_pixels[i]:y_checkpoint_pixels[i+1]])


In [5]:
pixels_of_block = []
for i in range(len(blockwise_pixels)):
    pixels_of_block.append(blockwise_pixels[i].flatten())

In [6]:
def s_calculator(pixels_in_a_block,block_number):
    cardinality_pixels_in_block = len(pixels_in_a_block)
    s=0
    for i in range(cardinality_pixels_in_block - 1):
       s = s + abs(pow(pixels_in_a_block[i],block_number)-pow(pixels_in_a_block[i+1],block_number))
    #print(s)
    s=s/(cardinality_pixels_in_block - 1)
    s=pow(s,1/block_number)
    #print(s)
    s=ceil(s)
    return s

In [7]:
s_values = []
for j in range(number_of_blocks):
    s_values.append(s_calculator(pixels_of_block[j],j+1))
print(s_values)

[9, 53, 77, 98, 101, 127, 127, 98, 154, 161, 156, 169, 175, 170, 173, 178]


In [8]:
def graph_maker(pixels_in_block,value_of_s):
    graph = {}
    cardinality_block = len(pixels_in_block)
    
    for i in range(cardinality_block):
        graph[i] = []
    for i in range(cardinality_block):
        for j in range(cardinality_block):
            if(i==j):
                continue
            elif(abs(int(pixels_in_block[i])-int(pixels_in_block[j]))<value_of_s):
                graph[i].append(j)
    return graph

In [9]:
graph_of_block = []
for k in range(number_of_blocks):
    graph_of_block.append(graph_maker(pixels_of_block[k],s_values[k]))

In [ ]:
def binary_to_message(binary_message):#will be a eight bit value
    

In [ ]:
def reveal_a_message(message,block_number):
    graph_dash = deepcopy(graph_of_block[block_number])
    print('copy successful')
    stop = False
    i=0
    c=0
    while not stop :
        #print('iteration ',c,'with i = ',i,end="")
        #c+=1
        if(len(graph_dash[i])>0):
            j = min(graph_dash[i])
            del graph_dash[j]
            for k in graph_dash:
                if(j in graph_dash[k]):
                    graph_dash[k].remove(j)
            #print(' and removed : ',j)
        else:
            q=[]
            for k in graph_dash:
                if k > i:
                    q.append(k)
            if(len(q)>0):
                i_dash = min(q)
                if(i_dash == max(graph_dash.keys())):
                    stop = True
                else:
                    i = i_dash
            else:
                stop = True
    
    print(2*len(graph_dash.keys()),' bits can be hidden')
    print('Nodes in color class : ',graph_dash.keys())
    new_pixels = deepcopy(pixels_of_block[block_number])
    #compare_print(new_pixels,pixels_of_block[block_number])
    for i in graph_dash.keys():
        c = len(message)
        if(c>0):
            pixel_value = bin(pixels_of_block[block_number][i])[2:]
            replacement = message[c-2:c]
            message = message[:c-2]
            c-=2
            print("IN ",pixel_value," last two are : ",pixel_value[-2:]," replaced by :",replacement)
            temp_pixel = ''
            temp_pixel+=pixel_value[:-2]
            temp_pixel+=replacement
            print(pixel_value," becomes : ",temp_pixel)
            for j in graph_of_block[block_number][i]:
                new_pixels[j]-=(int(pixel_value,2)-int(temp_pixel,2))
            new_pixels[i] -= (int(pixel_value,2)-int(temp_pixel,2)) 
            print(int(pixel_value,2)," becomes ",int(temp_pixel,2))
            #compare_print(new_pixels,pixels_of_block[block_number])
        else:
            print("\nmessage Finished")
            #compare_print(new_pixels,pixels_of_block[block_number])
            return new_pixels,1,message
    #compare_print(new_pixels,pixels_of_block[block_number])
    print('\nBlock Finished. Switching to new block')
    return new_pixels,0,message